In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from gensim.models import KeyedVectors

In [3]:
w2v=KeyedVectors.load("/content/gdrive/MyDrive/FYP/IMDb/Word2Vec/w2c")

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [7]:
import tensorflow as tf
import pickle
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  features=tokenizer.fit_on_texts(X)
  pickle.dump(features, open("tokenizer.pickle", "wb"))
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   19,  7608,  7149, ...,   123,  4013,   486],
       [  203,     2,    27, ...,  1900,    75,   216],
       [   10,   132,     3, ...,    67,    16,   333],
       ...,
       [ 3880,    70,   148, ..., 22558,     2,  5887],
       [   15,     1,   341, ...,    49,   695,    44],
       [    9,   154,    18, ...,   759,    11,    18]], dtype=int32)

In [8]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in w2v:
    embed_vector=w2v[word]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.17750642, -1.21290028, -0.95585269, ...,  0.432291  ,
        -0.217326  , -0.69201577],
       [ 1.5916748 ,  0.41039303,  0.79539949, ...,  1.64110196,
         0.89807343,  0.49340928],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)
model.save('/content/gdrive/MyDrive/W2VDNNmodel')

Epoch 1/100
586/586 [==============================] - 238s 404ms/step - loss: 0.6991 - accuracy: 0.5637
Epoch 2/100
586/586 [==============================] - 234s 400ms/step - loss: 0.6685 - accuracy: 0.5898
Epoch 3/100
586/586 [==============================] - 237s 404ms/step - loss: 0.6301 - accuracy: 0.6474
Epoch 4/100
586/586 [==============================] - 233s 397ms/step - loss: 0.5646 - accuracy: 0.7073
Epoch 5/100
586/586 [==============================] - 234s 399ms/step - loss: 0.5094 - accuracy: 0.7407
Epoch 6/100
586/586 [==============================] - 239s 408ms/step - loss: 0.4694 - accuracy: 0.7644
Epoch 7/100
586/586 [==============================] - 235s 401ms/step - loss: 0.4351 - accuracy: 0.7855
Epoch 8/100
586/586 [==============================] - 241s 412ms/step - loss: 0.4005 - accuracy: 0.8082
Epoch 9/100
586/586 [==============================] - 243s 415ms/step - loss: 0.3661 - accuracy: 0.8281
Epoch 10/100
586/586 [==============================] -

In [ ]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.8332
0.8331985385523154
0.8332934197290054
0.8333404687798907
